In [1]:
import os
import pandas as pd

# Load the metadata CSV file
df = pd.read_csv("HAM10000_metadata.csv")

# Part 1 and Part 2 directories
part1_dir = r'D:/archive (2)/HAM10000_images_part_1'
part2_dir = r'D:/archive (2)/HAM10000_images_part_2'

# Add image path from Part 1 or Part 2
def get_image_path(image_id):
    # Check if image exists in part 1
    image_path_part1 = os.path.join(part1_dir, f"{image_id}.jpg")
    if os.path.exists(image_path_part1):
        return image_path_part1
    # Check if image exists in part 2
    image_path_part2 = os.path.join(part2_dir, f"{image_id}.jpg")
    if os.path.exists(image_path_part2):
        return image_path_part2
    return None

# Add a new column 'image_path' with the full path to the image
df['image_path'] = df['image_id'].apply(get_image_path)

# Check if all images were successfully mapped
missing_images = df[df['image_path'].isnull()]
if not missing_images.empty:
    print(f"Missing images: {missing_images['image_id'].tolist()}")
else:
    print("All images successfully mapped!")


All images successfully mapped!


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Encode the labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['dx'])

# Split the dataset into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=101)

# Image data generators for augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create image generators
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw'
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw'
)

# Load the MobileNetV2 model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Add new layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(len(le.classes_), activation='softmax')(x)

# Build the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)

# Save the model in .h5 format
model.save("skin_disease_model.h5")

# Evaluate the model
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation accuracy: {val_accuracy:.4f}")


Found 8012 validated image filenames.
Found 2003 validated image filenames.
Epoch 1/10


C:\Users\Hidesh\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


251/251 ━━━━━━━━━━━━━━━━━━━━ 235s 880ms/step - accuracy: 0.6639 - loss: 1.0198 - val_accuracy: 0.7034 - val_loss: 0.8309
Epoch 2/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 634us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10


C:\Users\Hidesh\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


251/251 ━━━━━━━━━━━━━━━━━━━━ 206s 812ms/step - accuracy: 0.7248 - loss: 0.7771 - val_accuracy: 0.7039 - val_loss: 0.8261
Epoch 4/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 126us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 207s 816ms/step - accuracy: 0.7317 - loss: 0.7382 - val_accuracy: 0.7249 - val_loss: 0.7548
Epoch 6/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 122us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 205s 810ms/step - accuracy: 0.7355 - loss: 0.7211 - val_accuracy: 0.7294 - val_loss: 0.7888
Epoch 8/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 131us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 206s 812ms/step - accuracy: 0.7562 - loss: 0.6773 - val_accuracy: 0.7449 - val_loss: 0.7218
Epoch 10/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 116us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00


63/63 ━━━━━━━━━━━━━━━━━━━━ 32s 506ms/step - accuracy: 0.7360 - loss: 0.7368
Validation accuracy: 0.7449


In [3]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open('skin_disease_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model successfully converted to TensorFlow Lite!")


INFO:tensorflow:Assets written to: C:\Users\Hidesh\AppData\Local\Temp\tmpj3du87rj\assets


INFO:tensorflow:Assets written to: C:\Users\Hidesh\AppData\Local\Temp\tmpj3du87rj\assets


Saved artifact at 'C:\Users\Hidesh\AppData\Local\Temp\tmpj3du87rj'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  2999316912784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999316914320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999316913360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999316915088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999316912400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999316914704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999316914128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999316913552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999316912976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999316915664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999

In [4]:
# Convert the Keras model to TensorFlow Lite format
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open('final_model.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\Hidesh\AppData\Local\Temp\tmp_wqx9u78\assets


INFO:tensorflow:Assets written to: C:\Users\Hidesh\AppData\Local\Temp\tmp_wqx9u78\assets


Saved artifact at 'C:\Users\Hidesh\AppData\Local\Temp\tmp_wqx9u78'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  2999316912784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999316914320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999316913360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999316915088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999316912400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999316914704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999316914128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999316913552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999316912976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999316915664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2999